In [2]:
##IMPORTS
import sys
sys.path.append("/workspace/QuantLab")

import pandas as pd
from RQCharts.main import RQCharts
from RQAlpha import RTools
from talib import abstract
import matplotlib.pyplot as plt
import numpy as np
import random
from backtesting import Backtest
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
### Initial construction of random tickers list of df with indicators. 

indicators = []
resolution = "daily"
sma_ema_days = []
percent = 0.3
start_time = "2018-01-01"

RQCharts = RQCharts()
RTools = RTools()

stored_symbols = []
symbol_df_dict = {}

#IF resolution < Daily it will retrieve symbols already colelcted in minute data else in daily data
if resolution in ["minute", "5m", "10m", "15m", "30m", "hourly"]:
    db_response= RQCharts.conn.insert_new_command("SELECT DISTINCT symbol from stocks.minute;")
    for element in db_response:
        stored_symbols.append(element[0])
else:
    stored_symbols = RQCharts.watchlists.get_watched()


random_symbols = random.sample(stored_symbols, int(percent * len(stored_symbols))) 

if not any(ticker in random_symbols for ticker in ["SPY", "QQQ", "TSLA"]):
    missing_ticker = next(ticker for ticker in ["SPY", "QQQ", "TSLA"] if ticker not in random_symbols)
    random_symbols.append(missing_ticker) 

for symbol in random_symbols:
    start_time = start_time
    df = RQCharts.add_equity(symbol, resolution=resolution)
    df =df[start_time:]
    df['ATR'] = RTools.ATR(df, window = 20)
    df['RSI']= RTools.RSI(df, period =15)
    df['VWAP']= RTools.VWAP(df, window =15)
    RTools.BBANDS(df= df, window=20, std = 2.0)
    
    for days in sma_ema_days:
        df[f"EMA_{days}"] = RTools.EMA(df, window = days)
        df[f"SMA_{days}"] = RTools.SMA(df, window = days)
    df.dropna()
    df.dropna(subset=['Close'], inplace=True)  # Removes rows where 'close' is NaN
    df=df[df['High']!=df['Low']]
    
    symbol_df_dict[symbol]= df



In [4]:
#Signal STRATEGY: define your signal generating function here 

def total_signal(df):
    candle_signal = [0]*len(df)

    for current_pos in range(30,len(df)):

        # Buy condition
        c1 = df['High'].iloc[current_pos] > df['High'].iloc[current_pos-1]
        c2 = df['High'].iloc[current_pos-1] > df['Low'].iloc[current_pos]
        c3 = df['Low'].iloc[current_pos] > df['High'].iloc[current_pos-2]
        c4 = df['High'].iloc[current_pos-2] > df['Low'].iloc[current_pos-1]
        c5 = df['Low'].iloc[current_pos-1] > df['High'].iloc[current_pos-3]
        c6 = df['High'].iloc[current_pos-3] > df['Low'].iloc[current_pos-2]
        c7 = df['Low'].iloc[current_pos-2] > df['Low'].iloc[current_pos-3]

        if c1 and c2 and c3 and c4 and c5 and c6 and c7:
            candle_signal[current_pos] = 2
            
        # Symmetrical conditions for short (sell condition)
        c1 = df['Low'].iloc[current_pos] < df['Low'].iloc[current_pos-1]
        c2 = df['Low'].iloc[current_pos-1] < df['High'].iloc[current_pos]
        c3 = df['High'].iloc[current_pos] < df['Low'].iloc[current_pos-2]
        c4 = df['Low'].iloc[current_pos-2] < df['High'].iloc[current_pos-1]
        c5 = df['High'].iloc[current_pos-1] < df['Low'].iloc[current_pos-3]
        c6 = df['Low'].iloc[current_pos-3] < df['High'].iloc[current_pos-2]
        c7 = df['High'].iloc[current_pos-2] < df['High'].iloc[current_pos-3]

        if c1 and c2 and c3 and c4 and c5 and c6 and c7:
            candle_signal[current_pos] = 1
            
    df["TotalSignal"] = candle_signal

        
def add_pointpos_column(df, signal_column):
    """
    Adds a 'pointpos' column to the DataFrame to indicate the position of support and resistance points.
    
    Parameters:
    df (DataFrame): DataFrame containing the stock data with the specified SR column, 'Low', and 'High' columns.
    sr_column (str): The name of the column to consider for the SR (support/resistance) points.
    
    Returns:
    DataFrame: The original DataFrame with an additional 'pointpos' column.
    """
    def pointpos(row):
        if row[signal_column] == 2:
            return row['Low'] - 1e-4
        elif row[signal_column] == 1:
            return row['High'] + 1e-4
        else:
            return np.nan

    df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
    return df

def plot_candlestick_with_signals(df, start_index, num_rows):
    """
    Plots a candlestick chart with signal points.
    
    Parameters:
    df (DataFrame): DataFrame containing the stock data with 'Open', 'High', 'Low', 'Close', and 'pointpos' columns.
    start_index (int): The starting index for the subset of data to plot.
    num_rows (int): The number of rows of data to plot.
    
    Returns:
    None
    """
    df_subset = df[start_index:start_index + num_rows]
    
    fig = make_subplots(rows=1, cols=1)
    
    fig.add_trace(go.Candlestick(x=df_subset.index,
                                 open=df_subset['Open'],
                                 high=df_subset['High'],
                                 low=df_subset['Low'],
                                 close=df_subset['Close'],
                                 name='Candlesticks'),
                  row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df_subset.index, y=df_subset['pointpos'], mode="markers",
                             marker=dict(size=10, color="MediumPurple", symbol='circle'),
                             name="Entry Points"),
                  row=1, col=1)
    
    fig.update_layout(
        width=1200, 
        height=800, 
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False),
        showlegend=True,
        legend=dict(
            x=0.01,
            y=0.99,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="white"
            ),
            bgcolor="black",
            bordercolor="gray",
            borderwidth=2
        )
    )
    
    fig.show()


In [5]:
#FOLLOWUP

for i, df in symbol_df_dict.items():
    print("working on dataframe ", i, "...")
    total_signal(df)
    non_zero_count = (df["TotalSignal"] != 0).sum()
    
    add_pointpos_column(df, "TotalSignal")
    print(non_zero_count)

    #print (symbol_df_dict[i]["TotalSignal"])


working on dataframe  TSLA ...
58
working on dataframe  SMCI ...
40
working on dataframe  BITO ...
7
working on dataframe  XLRE ...
60
working on dataframe  GOOGL ...
49
working on dataframe  EWZ ...
24
working on dataframe  NKE ...
63
working on dataframe  EEM ...
24
working on dataframe  MARA ...
37
working on dataframe  SOFI ...
37
working on dataframe  SLV ...
30
working on dataframe  SOUN ...
20
working on dataframe  TQQQ ...
58
working on dataframe  XLC ...
62
working on dataframe  XLU ...
72
working on dataframe  BABA ...
38
working on dataframe  AVGO ...
73
working on dataframe  KRE ...
46
working on dataframe  HD ...
60
working on dataframe  MU ...
46


In [6]:
#plot_candlestick_with_signals(symbol_df_dict["SPY"], start_index=300, num_rows=355)

import matplotlib.pyplot as plt
for symbol, df in symbol_df_dict.items():
    print(df.isna().sum())
    print(np.isinf(df).sum())
    #df['Close'].plot(title=f'{symbol}')
    #plt.show()

Open              0
High              0
Low               0
Close             0
Volume            0
ATR               0
RSI              15
VWAP              0
upperband        19
lowerband        19
middleband       19
pband            19
wband            19
TotalSignal       0
pointpos       1699
dtype: int64
Open           0
High           0
Low            0
Close          0
Volume         0
ATR            0
RSI            0
VWAP           0
upperband      0
lowerband      0
middleband     0
pband          0
wband          0
TotalSignal    0
pointpos       0
dtype: int64
Open              0
High              0
Low               0
Close             0
Volume            0
ATR               0
RSI              15
VWAP              0
upperband        19
lowerband        19
middleband       19
pband            19
wband            19
TotalSignal       0
pointpos       1712
dtype: int64
Open           0
High           0
Low            0
Close          0
Volume         0
ATR            0
RSI 

In [11]:
#Backtesting Layout: define your backtesting class here
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL():
    return df.TotalSignal

class ThreeCandleRejectionStrat(Strategy):
    mysize = 0.2
    slperc = 0.05
    tpperc = 0.03

    def init(self):
        super().init()  
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        if self.signal1 ==2 and len(self.trades) < 3:
            current_close = self.data.Close[-1]
            sl = current_close - self.slperc * current_close
            tp = current_close + self.tpperc * current_close
            self.buy(size=self.mysize, sl = sl, tp =tp)

        elif self.signal1 ==1 and len(self.trades) < 3:
            current_close = self.data.Close[-1]
            sl = current_close + self.slperc * current_close
            tp = current_close - self.tpperc * current_close
            self.sell(size = self.mysize, sl=sl , tp=tp)


In [12]:
## Run Optimize and sort your Backtest results
strat_name = "ThreeCandleRejectionStrat"

cash = 10000
margin = 1/10
commission= .01
results_df = {}  

for symbol, df in symbol_df_dict.items(): 
    df = df[20:] 
    df.dropna()
    print(f"Running backtest for {symbol}...")  

    try:
        # Run the backtest with optimization
        bt = Backtest(
            df, 
            ThreeCandleRejectionStrat, 
            cash=cash, 
            margin=margin, 
            commission=commission
        )
        
        # Optimize parameters
        stat, heatmap = bt.optimize(
            tpperc=np.arange(0.01, .5, 0.09).tolist(),
            slperc=np.arange(0.01, .5, 0.09).tolist(),
            maximize="Sharpe Ratio",
            method="grid",
            return_heatmap=True,
            return_optimization=False,
            random_state=None,
        )

        # Plot backtest results
        bt.plot(
            filename=f"Backtest_Results/{symbol}_{strat_name}.html",
            plot_equity=True,
            plot_pl=True,
            plot_volume=True,
            plot_trades=True,
            resample=True,
            open_browser=False,
        )
    
    except OverflowError as e:
        print(f"OverflowError for {symbol}: {e}. Skipping to the next symbol.")
    except Exception as e:
        print(f"An unexpected error occurred for {symbol}: {e}. Skipping to the next symbol.")
    # Store results
    results_df[symbol] = {"stats": stat, "heatmap": heatmap}


Running backtest for TSLA...
Running backtest for SMCI...
Running backtest for BITO...
Running backtest for XLRE...
Running backtest for GOOGL...
Running backtest for EWZ...
Running backtest for NKE...
Running backtest for EEM...
Running backtest for MARA...
Running backtest for SOFI...
Running backtest for SLV...
Running backtest for SOUN...
Running backtest for TQQQ...
Running backtest for XLC...
Running backtest for XLU...
Running backtest for BABA...
Running backtest for AVGO...
Running backtest for KRE...
Running backtest for HD...
Running backtest for MU...


In [13]:
#ANALYZE RETURNS
# Initialize an empty list to store summary metrics for each symbol
summary_stats = []

# Analyze each symbol's stats and heatmap
for symbol, result in results_df.items():
    stats = result["stats"]
    heatmap = result["heatmap"]

    # Extract important metrics from stats
    metrics = {
    "Symbol": symbol,
    "Start": stats["Start"],
    "End": stats["End"],
    "Duration": stats["Duration"],
    "Exposure Time [%]": stats["Exposure Time [%]"],
    "Equity Final [$]": stats["Equity Final [$]"],
    "Equity Peak [$]": stats["Equity Peak [$]"],
    "Return [%]": stats["Return [%]"],
    "Buy & Hold Return [%]": stats["Buy & Hold Return [%]"],
    "Return (Ann.) [%]": stats["Return (Ann.) [%]"],
    "Volatility (Ann.) [%]": stats["Volatility (Ann.) [%]"],
    "Sharpe Ratio": stats["Sharpe Ratio"],
    "Max. Drawdown [%]": stats["Max. Drawdown [%]"],
    "Trades": stats["# Trades"],
    "Win Rate [%]": stats["Win Rate [%]"],
    "Avg. Trade [%]": stats["Avg. Trade [%]"],
    "Avg. Trade Duration": stats["Avg. Trade Duration"],
    "Profit Factor": stats["Profit Factor"],
    "Expectancy [%]": stats["Expectancy [%]"]
}

    

    # Append metrics to the list
    summary_stats.append(metrics)

# Convert the summary_stats list to a DataFrame
summary_stats_df = pd.DataFrame(summary_stats)

# Analyze the summary_stats DataFrame
#print("\nSummary of all symbols:")
#print(summary_stats_df)

# Sort by Sharpe Ratio for the best strategies
best_strategies = summary_stats_df.sort_values(by="Sharpe Ratio", ascending=False)
print("\nTop Strategies by Sharpe Ratio:")
print(best_strategies.head(10))

# Save summary stats to CSV
summary_stats_df.to_csv("strategy_summary.csv", index=False)
print("\nSummary stats saved to 'strategy_summary.csv'.")





Top Strategies by Sharpe Ratio:
   Symbol      Start        End  Duration  Exposure Time [%]  \
14    XLU 2018-01-31 2024-12-23 2518 days          92.569124   
11   SOUN 2022-05-26 2024-12-23  942 days           7.109737   
8    MARA 2018-01-31 2024-12-23 2518 days           5.472350   
5     EWZ 2018-01-31 2024-12-23 2518 days          50.921659   
3    XLRE 2018-01-31 2025-02-10 2567 days          29.485003   
2    BITO 2021-11-16 2024-12-23 1133 days          46.282051   
1    SMCI 2018-01-31 2024-12-23 2518 days           1.674365   
0    TSLA 2018-01-31 2024-12-24 2519 days          23.891767   
7     EEM 2018-01-31 2024-12-23 2518 days          22.638249   
6     NKE 2018-01-31 2024-12-23 2518 days          46.198157   

    Equity Final [$]  Equity Peak [$]  Return [%]  Buy & Hold Return [%]  \
14      92310.655248    124826.365248  823.106552              49.020376   
11      21645.628628     21645.628628  116.456286             213.343558   
8       14732.559130     16025.078